In [ ]:
import tensorflow as tf
import pandas as pd

from stable_baselines.common.policies import MlpLnLstmPolicy
from stable_baselines import PPO2

from tensortrade.strategies import StableBaselinesTradingStrategy
from tensortrade.environments import TradingEnvironment
from tensortrade.rewards import RiskAdjustedReturns
from tensortrade.actions import ManagedRiskOrders
from tensortrade.instruments import Quantity, TradingPair, BTC, USD
from tensortrade.orders.criteria import StopLossCriteria, StopDirection
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.exchanges.simulated import SimulatedExchange
from tensortrade.features.stationarity import LogDifference
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline

WINDOW_SIZE = 1
PRICE_COLUMN = 'close'

normalize = MinMaxNormalizer(inplace=True)
difference = LogDifference(inplace=True)
feature_pipeline = FeaturePipeline(steps=[normalize])

action_scheme = ManagedRiskOrders(pairs=[USD/BTC])
reward_scheme = RiskAdjustedReturns(return_algorithm="sortino")

csv_file = tf.keras.utils.get_file(
    'Coinbase_BTCUSD_1h.csv', 'https://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv')
ohlcv_data = pd.read_csv(csv_file, skiprows=1, index_col="Date")
ohlcv_data.columns = map(str.lower, ohlcv_data.columns)
ohlcv_data = ohlcv_data.rename(columns={'volume btc': 'volume'})

# ohlcv_data = pd.read_csv('./data/updateOHLCdata_master.csv')

exchange = SimulatedExchange(data_frame=ohlcv_data, price_column=PRICE_COLUMN, randomize_time_slices=True)

wallets = [(exchange, USD, 10000), (exchange, BTC, 0)]

portfolio = Portfolio(base_instrument=USD, wallet_tuples=wallets)

environment = TradingEnvironment(exchange=exchange,
                                 portfolio=portfolio,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme,
                                 feature_pipeline=feature_pipeline,
                                 window_size=WINDOW_SIZE,
                                 observe_wallets=[USD, BTC])

print('Observation Data:')
print(environment.observation_columns)

model = PPO2
policy = MlpLnLstmPolicy
params = { "learning_rate": 1e-5, 'nminibatches': 1 }

strategy = StableBaselinesTradingStrategy(environment=environment,
                                          model=model,
                                          policy=policy,
                                          model_kwargs=params)

strategy.run(steps=50000)

W1210 14:49:22.142772 4441394624 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/module.py:31: The name tf.enable_resource_variables is deprecated. Please use tf.compat.v1.enable_resource_variables instead.

W1210 14:49:22.156360 4441394624 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:46: The name tf.train.AdadeltaOptimizer is deprecated. Please use tf.compat.v1.train.AdadeltaOptimizer instead.

W1210 14:49:22.157423 4441394624 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:47: The name tf.train.AdagradOptimizer is deprecated. Please use tf.compat.v1.train.AdagradOptimizer instead.

W1210 14:49:22.158410 4441394624 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:48: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train

Observation Data:
['open' 'high' 'low' 'close' 'volume' 'volume usd' 'USD' 'BTC'
 'USD_locked' 'BTC_locked']


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
I1210 14:49:51.746127 4441394624 trading_environment.py:357] Order: 1578fe49-8920-4c48-b84f-18ad38572243 | OrderStatus.FILLED | TradeSide.BUY | TradeType.MARKET | USD/BTC | 3000.00 USD | 11622.61 | None -> a117b0a8-f5ef-4770-8

Fill:  1578fe49-8920-4c48-b84f-18ad38572243 -> 1 | TradeSide.BUY | TradeType.MARKET | USD/BTC | Size: 2991.0 | Price: 11667.24 | Commission: 9.0
Wallet Balance: 0.25635883 BTC | Locked: 0.00000000 BTC

Fill:  394036f1-ddea-478e-a772-b1b35127f9e3 -> 2 | TradeSide.BUY | TradeType.MARKET | USD/BTC | Size: 2093.7 | Price: 11720.02 | Commission: 6.3
Wallet Balance: 0.17864304 BTC | Locked: 0.25635883 BTC

Fill:  94d7b012-4615-4b09-bddb-e8e206af59cf -> 3 | TradeSide.BUY | TradeType.MARKET | USD/BTC | Size: 4885.3 | Price: 11573.56 | Commission: 14.7
Wallet Balance: 0.42210867 BTC | Locked: 0.43500187 BTC



I1210 14:49:51.948388 4441394624 trading_environment.py:357] Order: None
I1210 14:49:51.950008 4441394624 trading_environment.py:358] Observation: [2.30201000e-05 2.33572000e-05 2.28581000e-05 2.32669000e-05
 7.25380000e-06 1.67842429e-02 1.00000000e-16 1.00000000e-16
 1.00000000e-16 8.57110550e-09]
I1210 14:49:51.951150 4441394624 trading_environment.py:359] P/L: 0.99861177615914
I1210 14:49:51.951936 4441394624 trading_environment.py:360] Reward (6): -0.5511160474229854
I1210 14:49:51.956186 4441394624 trading_environment.py:361] Performance:    BTC  BTC_locked  USD  USD_locked    net_worth step
0  0.0    0.857111  0.0         0.0  9733.458717    5
I1210 14:49:51.989245 4441394624 trading_environment.py:357] Order: None
I1210 14:49:51.990346 4441394624 trading_environment.py:358] Observation: [2.3541900e-05 2.3541900e-05 2.2780000e-05 2.3020100e-05 1.4396800e-05
 3.3284565e-02 1.0000000e-16 1.0000000e-16 1.0000000e-16 8.5711055e-09]
I1210 14:49:51.991823 4441394624 trading_environmen

...

I1210 14:51:28.919265 4441394624 trading_environment.py:361] Performance:         BTC  BTC_locked  USD  USD_locked   net_worth  step
0  0.504601         0.0  0.0         0.0  3960.92294  1984
I1210 14:51:28.952409 4441394624 trading_environment.py:357] Order: None
I1210 14:51:28.953464 4441394624 trading_environment.py:358] Observation: [3.9850000e-05 3.9854500e-05 3.9837700e-05 3.9837800e-05 1.7843000e-06
 7.1095978e-03 1.0000000e-16 5.0460125e-09 1.0000000e-16 1.0000000e-16]
I1210 14:51:28.954356 4441394624 trading_environment.py:359] P/L: 0.40260518215631996
I1210 14:51:28.954890 4441394624 trading_environment.py:360] Reward (1986): -0.005711758483926898
I1210 14:51:28.958437 4441394624 trading_environment.py:361] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth  step
0  0.504601         0.0  0.0         0.0  4024.003141  1985
I1210 14:51:28.990391 4441394624 trading_environment.py:357] Order: None
I1210 14:51:28.991641 4441394624 trading_environment.py:358] Observ

I1210 14:51:29.395467 4441394624 trading_environment.py:357] Order: None
I1210 14:51:29.396343 4441394624 trading_environment.py:358] Observation: [3.9800100e-05 3.9829900e-05 3.9758000e-05 3.9827600e-05 1.2182000e-06
 4.8484629e-03 1.0000000e-16 5.0460125e-09 1.0000000e-16 1.0000000e-16]
I1210 14:51:29.397126 4441394624 trading_environment.py:359] P/L: 0.41246609958839997
I1210 14:51:29.397657 4441394624 trading_environment.py:360] Reward (1998): -0.005598555096708425
I1210 14:51:29.400922 4441394624 trading_environment.py:361] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth  step
0  0.504601         0.0  0.0         0.0  4112.959293  1997
I1210 14:51:29.432250 4441394624 trading_environment.py:357] Order: None
I1210 14:51:29.433217 4441394624 trading_environment.py:358] Observation: [3.9731200e-05 3.9816700e-05 3.9731100e-05 3.9800100e-05 8.2200000e-07
 3.2702631e-03 1.0000000e-16 5.0460125e-09 1.0000000e-16 1.0000000e-16]
I1210 14:51:29.433943 4441394624 trading_e

I1210 14:51:30.119462 4441394624 trading_environment.py:359] P/L: 0.39797950258924
I1210 14:51:30.120098 4441394624 trading_environment.py:360] Reward (2010): -0.005606033516803829
I1210 14:51:30.123922 4441394624 trading_environment.py:361] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth  step
0  0.504601         0.0  0.0         0.0  4079.665703  2009
I1210 14:51:30.156449 4441394624 trading_environment.py:357] Order: None
I1210 14:51:30.157453 4441394624 trading_environment.py:358] Observation: [3.97998000e-05 3.98600000e-05 3.97500000e-05 3.97500000e-05
 4.32280000e-06 1.72086272e-02 1.00000000e-16 5.04601250e-09
 1.00000000e-16 1.00000000e-16]
I1210 14:51:30.158266 4441394624 trading_environment.py:359] P/L: 0.3992692633586799
I1210 14:51:30.158817 4441394624 trading_environment.py:360] Reward (2011): -0.005701634570263439
I1210 14:51:30.162042 4441394624 trading_environment.py:361] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth  step
0  0.50

I1210 14:51:30.564397 4441394624 trading_environment.py:361] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth  step
0  0.504601         0.0  0.0         0.0  3925.565531  2021
I1210 14:51:30.595691 4441394624 trading_environment.py:357] Order: None
I1210 14:51:30.596562 4441394624 trading_environment.py:358] Observation: [4.0300000e-05 4.0300000e-05 4.0273800e-05 4.0273900e-05 9.3020000e-07
 3.7479664e-03 1.0000000e-16 5.0460125e-09 1.0000000e-16 1.0000000e-16]
I1210 14:51:30.597280 4441394624 trading_environment.py:359] P/L: 0.39257270030263997
I1210 14:51:30.597991 4441394624 trading_environment.py:360] Reward (2023): -0.005757281925885999
I1210 14:51:30.601498 4441394624 trading_environment.py:361] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth  step
0  0.504601         0.0  0.0         0.0  3896.571143  2022
I1210 14:51:30.633388 4441394624 trading_environment.py:357] Order: None
I1210 14:51:30.634335 4441394624 trading_environment.py:358] Obse

I1210 14:51:31.043621 4441394624 trading_environment.py:357] Order: None
I1210 14:51:31.044867 4441394624 trading_environment.py:358] Observation: [4.00850000e-05 4.04482000e-05 4.00849000e-05 4.03033000e-05
 7.75840000e-06 3.12223696e-02 1.00000000e-16 5.04601250e-09
 1.00000000e-16 1.00000000e-16]
I1210 14:51:31.045761 4441394624 trading_environment.py:359] P/L: 0.4200805323
I1210 14:51:31.046283 4441394624 trading_environment.py:360] Reward (2035): -0.005602309075028872
I1210 14:51:31.050681 4441394624 trading_environment.py:361] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth  step
0  0.504601         0.0  0.0         0.0  4030.118908  2034
I1210 14:51:31.083692 4441394624 trading_environment.py:357] Order: None
I1210 14:51:31.084603 4441394624 trading_environment.py:358] Observation: [4.01145000e-05 4.01244000e-05 4.00849000e-05 4.00850000e-05
 2.76920000e-06 1.11072031e-02 1.00000000e-16 5.04601250e-09
 1.00000000e-16 1.00000000e-16]
I1210 14:51:31.085266 44413